<a href="https://colab.research.google.com/github/kosar-MRosh/Signals-Processing-by-Machine-Learning-/blob/main/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Necessary Modules

In [ ]:
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Input, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from os import listdir
from glob import glob
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import keras
import os
import random
import cv2
import imutils
import PIL

%matplotlib inline

**Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = "/content/drive/MyDrive/MachineLearning/Datasets/Train/"

batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3000 files belonging to 2 classes.
Using 2700 files for training.
Found 3000 files belonging to 2 classes.
Using 300 files for validation.


In [ ]:
class_names = train_ds.class_names
print('class names =',class_names)
num_classes = len(class_names)
print('number of classes =', num_classes)

class names = ['GBM', 'LGG']
number of classes = 2


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

image_batch, labels_batch = next(iter(train_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.8847371


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
inception = InceptionV3(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

#Here we freeze the last 4 layers
#Layers are set to trainable as true by default

for layer in inception.layers:
    layer.trainable = False

for(i, layer) in enumerate(inception.layers):
   print(str(i)+" "+layer.__class__.__name__,layer.trainable)


87910968/87910968 [==============================] - 0s 0us/step
0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 Activation False
4 Conv2D False
5 BatchNormalization False
6 Activation False
7 Conv2D False
8 BatchNormalization False
9 Activation False
10 MaxPooling2D False
11 Conv2D False
12 BatchNormalization False
13 Activation False
14 Conv2D False
15 BatchNormalization False
16 Activation False
17 MaxPooling2D False
18 Conv2D False
19 BatchNormalization False
20 Activation False
21 Conv2D False
22 Conv2D False
23 BatchNormalization False
24 BatchNormalization False
25 Activation False
26 Activation False
27 AveragePooling2D False
28 Conv2D False
29 Conv2D False
30 Conv2D False
31 Conv2D False
32 BatchNormalization False
33 BatchNormalization False
34 BatchNormalization False
35 BatchNormalization False
36 Activation False
37 Activation False
38 Activation False
39 Activation False
40 Concatenate False
41 Conv2D False
42 BatchNormalization False
43 Activation False
44

In [ ]:
from keras.layers.pooling.global_average_pooling1d import GlobalAveragePooling1D
from keras.models import sequential
from keras.models import convolution2D, Dropout, Dense, MaxPooling2D
from keras.models import BachNormalization
from keras.models import MaxPooling2D
from keras.models import Functional
from keras.models import regularizers

#creates the top or head of the model that will be paced ontop of the bottom layers

def addTopModelinception(bottom_model, num_classes):

    """creates the top or head of the model that will be paced ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling1D()(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(512, activation = 'relu')(top_model)
    top_model = Dense(128, activation = 'relu', kernel_regularizers = regularizers.l2(0.01))(top_model)
    top_model = Dense(num_classes = 2, activation = 'sigmoid')(top_model)
    return top_model

SyntaxError: ignored

In [ ]:
from keras.layers.serialization import activation
from keras.models import sequential
from keras.layers import Dense, Dropout, activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.models import Model

In [ ]:
, output
num_classes = 2
FC_Head = addTopModelinception(inception, num_classes)
model = Model(inputs = vgg.Input, output = FC_Head)

print(model.summary())